In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta
import json
import pyspark.sql.functions

## Load variables from key vault

In [3]:
kv_scope = 'key-vault-secret'

# Variables
storage_account_name = dbutils.secrets.get(scope=kv_scope, key='traffic-storage-accountname') 
storage_account_access_key = dbutils.secrets.get(scope=kv_scope, key='traffic-storage-accountkey') 
eventgrid_accesskey = dbutils.secrets.get(scope=kv_scope, key='traffic-eventgrid-accesskey') 
eventgrid_topic = dbutils.secrets.get(scope=kv_scope, key='traffic-eventgrid-topicendpoint')

In [4]:
traffic_table_name = 'CameraTelemetry' + datetime.today().strftime('%Y%m%d')
speed_table_name = 'SpeedMeasurements' + datetime.today().strftime('%Y%m%d')

## Mounting the segment configuration json from blob

- Using the mount functionality to load the blob file

In [6]:
mount_name = 'traffic-config'
to_be_mounted = True
mounts = dbutils.fs.ls('/mnt/')

for mnt in mounts:
  if mnt.name.startswith(mount_name):
    to_be_mounted = False

if to_be_mounted:
  dbutils.fs.mount(
  source = 'wasbs://traffic-config@' + storage_account_name + '.blob.core.windows.net',
  mount_point = '/mnt/' + mount_name,
  extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net':storage_account_access_key})
else:
  print('Traffic config already mounted')

Traffic config already mounted

## Parsing segment configuration

- Reading the json file (`multiLine=True` !!)
- Adding calculated field for maximum duration (`(distance / speedlimit) * 3.6`), where 3.6 is coming from meters/second
- Only returning the relevant fields for the calculation query

In [8]:
segment_config = spark.read.json('/mnt/' + mount_name, multiLine=True) \
  .withColumn('TrajectId', col('segmentId')) \
  .withColumn('MinDuration', ((col('cameraDistance') / col('speedLimit')) * 3.6)) \
  .select('TrajectId','MinDuration', 'CameraDistance', 'SpeedLimit')
display(segment_config)

TrajectId,MinDuration,CameraDistance,SpeedLimit
dev,80.0,2000,90
edge01,80.0,2000,90
01,80.0,2000,90
02,150.0,5000,120
03,261.81818181818187,8000,110
04,49.09090909090909,1500,110


In [9]:
timestamp_from = datetime.utcnow() - timedelta(hours=0, minutes=20)
print(timestamp_from)

2020-05-27 07:54:23.686048

In [10]:
cameraStream = spark.readStream.format('delta') \
  .table(traffic_table_name) \
  .where(col('EventTime')>'2019-12-12 18:30:00') 

## Query that shows traffic (car count) per 5 second window

In [12]:
import pyspark.sql.functions as F

traffic_df = cameraStream \
  .where(col('EventTime') > timestamp_from) \
  .groupBy(window('EventTime', '30 seconds')) \
  .agg(count('*').alias('count')) \
  .select(col('window.start').alias('start'), 'count')

display(traffic_df)

start,count
2020-05-27T08:13:00.000+0000,35
2020-05-27T08:16:00.000+0000,11
2020-05-27T08:15:00.000+0000,12
2020-05-27T08:11:00.000+0000,112
2020-05-27T08:17:00.000+0000,14
2020-05-27T08:14:30.000+0000,12
2020-05-27T08:14:00.000+0000,12
2020-05-27T08:10:30.000+0000,84
2020-05-27T08:12:00.000+0000,124
2020-05-27T08:17:30.000+0000,4


## Query that shows the number of cars, grouped by make

In [14]:
make_df = cameraStream \
  .groupBy('Make', window('EventTime', '30 seconds')) \
  .agg(count('*').alias('count')) \
  .sort(col('count').desc()) \
  .select(col('window.start').alias('start'), 'Make', 'count')

display(make_df)

start,Make,count
2020-05-27T08:11:00.000+0000,Audi,16
2020-05-27T08:11:00.000+0000,Toyota,16
2020-05-27T08:12:00.000+0000,Suzuki,16
2020-05-27T08:12:00.000+0000,Opel,16
2020-05-27T08:11:00.000+0000,Mercedes,16
2020-05-27T08:11:30.000+0000,Saab,15
2020-05-27T08:12:00.000+0000,Volkswagen,15
2020-05-27T08:11:30.000+0000,Audi,14
2020-05-27T08:11:30.000+0000,Suzuki,14
2020-05-27T08:10:30.000+0000,Saab,13


## Query that shows number of cars per segment

In [16]:
segment_df = cameraStream \
  .groupBy('TrajectId', window('EventTime', '60 seconds')) \
  .agg(count('*').alias('count')) \
  .sort(col('count').desc()) \
  .select('TrajectId', col('window.start').alias('WindowTime'), 'count')

display(segment_df)

## Query that shows the number of cars, grouped by country

In [18]:
import pyspark.sql.functions as F

country_df = cameraStream \
  .groupBy('Country') \
  .agg(count('*').alias('count')) \
  .sort(F.col('count').desc()) \
  .select('Country', 'count')

display(country_df)

Country,count
BE,508
NL,47
DE,45
FR,32
PL,28
HU,8
PT,1


## Speed measurements & reports

In [20]:
speedMeasurementStream = spark.readStream.option('ignoreChanges', 'true').table(speed_table_name)

In [21]:
t02 = speedMeasurementStream.where(col('LastEvent')>'2020-05-26 19:20:00') 
display(t02)

TrajectId,LicensePlate,Speed,Make,Country,LastEvent,SpeedLimit,Duration
01,1-AGI-045,135.8490566037736,Mercedes,BE,2020-05-27T08:11:45.131+0000,90,53
04,1-ARM-623,83.07692307692308,Suzuki,BE,2020-05-27T08:11:52.189+0000,110,65
04,1-WVT-015,85.71428571428572,Volvo,PL,2020-05-27T08:11:45.967+0000,110,63
01,1-FOT-659,120.00000000000001,Audi,BE,2020-05-27T08:11:55.415+0000,90,60
01,1-DBO-791,122.03389830508476,Saab,FR,2020-05-27T08:11:39.176+0000,90,59
01,1-UBR-317,116.12903225806451,Saab,BE,2020-05-27T08:11:50.353+0000,90,62
04,1-DRF-450,85.71428571428572,BMW,BE,2020-05-27T08:11:55.309+0000,110,63
01,1-TEZ-181,120.00000000000001,Volkswagen,BE,2020-05-27T08:11:46.484+0000,90,60
01,1-EJV-625,124.13793103448278,Renault,DE,2020-05-27T08:12:01.728+0000,90,58
01,1-ZZD-915,116.12903225806451,Suzuki,NL,2020-05-27T08:12:02.772+0000,90,62


In [22]:
#avg_speed_df = speedMeasurementStream \
#  .groupBy('TrajectId', window('LastEvent', '60 seconds')) \
#  .agg(avg('Speed').alias('Speed')) \
#  .select('TrajectId', col('window.start').alias('WindowTime'), 'Speed')
avg_speed_df = speedMeasurementStream \
  .groupBy('TrajectId', window('LastEvent', '60 seconds')) \
  .agg(avg('Duration').alias('Duration')) \
  .select('TrajectId', col('window.start').alias('WindowTime'), 'Duration')

display(avg_speed_df)

TrajectId,WindowTime,Duration
01,2020-05-27T08:12:00.000+0000,57.653753026634384
02,2020-05-27T08:12:00.000+0000,92.57142857142857
04,2020-05-27T08:12:00.000+0000,61.10344827586207
03,2020-05-27T08:12:00.000+0000,62.964
04,2020-05-27T08:11:00.000+0000,63.666666666666664
03,2020-05-27T08:11:00.000+0000,62.122950819672134
01,2020-05-27T08:13:00.000+0000,55.07936507936508
01,2020-05-27T08:11:00.000+0000,58.166666666666664
04,2020-05-27T08:13:00.000+0000,65.76119402985074


In [23]:
avg_speed_bymake_df = speedMeasurementStream \
  .groupBy('Make', window('LastEvent', '30 seconds')) \
  .agg(avg('Speed').alias('Speed')) \
  .select('Make', col('window.start').alias('WindowTime'), 'Speed')

display(avg_speed_bymake_df)

Make,WindowTime,Speed
Mercedes,2020-05-27T08:11:30.000+0000,135.8490566037736
Opel,2020-05-27T08:13:30.000+0000,124.13793103448278
Suzuki,2020-05-27T08:12:30.000+0000,224.8665141113654
Opel,2020-05-27T08:12:30.000+0000,148.52738336713998
Saab,2020-05-27T08:12:00.000+0000,443.0769230769231
BMW,2020-05-27T08:11:30.000+0000,279.3522267206478
Renault,2020-05-27T08:13:00.000+0000,94.05314685314687
Opel,2020-05-27T08:12:00.000+0000,464.51612903225805
Volvo,2020-05-27T08:12:30.000+0000,128.57142857142858
Volkswagen,2020-05-27T08:13:30.000+0000,79.6875
